In [ ]:
import pathlib
import requests
import urllib
import dotenv
import os
import shapely
import shapely.geometry
import geopandas

In [ ]:
SCHEME = "https"
NETLOC_API = "datafinder.stats.govt.nz"
WFS_PATH_API_START = "/services;key="
WFS_PATH_API_END = "/wfs"

dotenv.load_dotenv()
KEY = os.environ.get('STATSNZ_API', None)

CRS = "EPSG:2193"

data_url = urllib.parse.urlunparse((SCHEME, NETLOC_API, f"{WFS_PATH_API_START}{KEY}{WFS_PATH_API_END}", "", "", ""))

In [ ]:
KEY

In [ ]:
x0 = 1752000
x1 = 1753000
y0 = 5430000
y1 = 5440000

In [ ]:
bbox = shapely.geometry.Polygon([(x0, y0), (x0, y1), (x1, y1), (x1, y0)])
bbox = geopandas.GeoSeries([bbox])
bbox = bbox.set_crs(CRS)

In [ ]:
def make_api_params(layer, geometry_type):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}",
                "cql_filter": f"bbox({geometry_type}, {y0}, {x0}, " +
                              f"{y1}, {x1}, " +
                              f"'urn:ogc:def:crs:{CRS}')"
            }
    return api_query

In [ ]:
def make_api_params_no_bounds(layer):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}"
            }
    return api_query

In [ ]:
def get_features_in_bounds(json_response, bbox):
    crs = json_response['crs']['properties']['name']

    # Cycle through each feature checking in bounds and getting geometry and properties
    features = {'geometry': []}
    for feature in json_response['features']:

        shapely_geometry = shapely.geometry.shape(feature['geometry'])

        # check intersection of tile and catchment in LINZ CRS
        if bbox.intersects(shapely_geometry).any():

            # Create column headings for each 'properties' option from the first in-bounds vector
            if len(features['geometry']) == 0:
                for key in feature['properties'].keys():
                    features[key] = []  # The empty list to append the property values too

            # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
            if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
                shapely_geometry = shapely_geometry[0]
            features['geometry'].append(shapely_geometry)

            # Add the value of each property in turn
            for key in feature['properties'].keys():
                features[key].append(feature['properties'][key])

    # Convert to a geopandas dataframe
    if len(features) > 0:
        features = geopandas.GeoDataFrame(features, crs=crs)
    else:
        features = None
    return features

In [ ]:
def get_features_no_bounds(json_response):
    crs = json_response['crs']['properties']['name']

    # Cycle through each feature checking in bounds and getting geometry and properties
    features = {'geometry': []}
    for feature in json_response['features']:

        shapely_geometry = shapely.geometry.shape(feature['geometry'])

        # Create column headings for each 'properties' option from the first in-bounds vector
        if len(features['geometry']) == 0:
            for key in feature['properties'].keys():
                features[key] = []  # The empty list to append the property values too

        # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
        if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
            shapely_geometry = shapely_geometry[0]
        features['geometry'].append(shapely_geometry)

        # Add the value of each property in turn
        for key in feature['properties'].keys():
            features[key].append(feature['properties'][key])

    # Convert to a geopandas dataframe
    if len(features) > 0:
        features = geopandas.GeoDataFrame(features, crs=crs)
    else:
        features = None
    return features

## Regional council Boundaries

In [ ]:
layer = 105133
geom_type = "Shape"

params_in_bounds = make_api_params(layer, geom_type)
params_no_bounds = make_api_params_no_bounds(layer)

print(requests.Request('POST', data_url, params=params_in_bounds).prepare().url)
print(requests.Request('POST', data_url, params=params_no_bounds).prepare().url)

response_in_bounds = requests.get(data_url, params=params_in_bounds, stream=True)
response_in_bounds.raise_for_status()
json_response_in_bounds=response_in_bounds.json()

response_no_bounds = requests.get(data_url, params=params_no_bounds, stream=True)
response_no_bounds.raise_for_status()
json_response_no_bounds=response_no_bounds.json()

features_no_bounds = get_features_no_bounds(json_response_no_bounds)
features_in_bounds = get_features_in_bounds(json_response_in_bounds, bbox)

In [ ]:
[features_no_bounds.geometry.area.sum(), features_no_bounds.loc[0].geometry.geometryType(), 
 features_no_bounds.geometry.length.sum(), features_no_bounds.columns, list(features_no_bounds['AREA_SQ_KM'][0:5])]

In [ ]:
[features_in_bounds.geometry.area.sum(), features_in_bounds.geometry.length.sum(), features_in_bounds.columns, 
 features_in_bounds.loc[0].geometry.geometryType(), list(features_in_bounds['AREA_SQ_KM'][0:5])]

# District health board 2015

In [ ]:
layer = 87883
geom_type = "GEOMETRY"

params_in_bounds = make_api_params(layer, geom_type)
params_no_bounds = make_api_params_no_bounds(layer)

print(requests.Request('POST', data_url, params=params_in_bounds).prepare().url)
print(requests.Request('POST', data_url, params=params_no_bounds).prepare().url)

response_in_bounds = requests.get(data_url, params=params_in_bounds, stream=True)
response_in_bounds.raise_for_status()
json_response_in_bounds=response_in_bounds.json()

features_in_bounds = get_features_in_bounds(json_response_in_bounds, bbox)

response_no_bounds = requests.get(data_url, params=params_no_bounds, stream=True)
response_no_bounds.raise_for_status()
json_response_no_bounds=response_no_bounds.json()

features_no_bounds = get_features_no_bounds(json_response_no_bounds)


In [ ]:
[features_no_bounds.geometry.area.sum(), features_no_bounds.loc[0].geometry.geometryType(), 
 features_no_bounds.geometry.length.sum(), features_no_bounds.columns, list(features_no_bounds['DHB2015_Code'][0:5])]

In [ ]:
[features_in_bounds.geometry.area.sum(), features_in_bounds.loc[0].geometry.geometryType(), 
 features_in_bounds.geometry.length.sum(), features_in_bounds.columns, list(features_in_bounds['DHB2015_Code'][0:5])]